<a href="https://colab.research.google.com/github/healthonrails/annolid/blob/master/docs/tutorials/Annolid_video_batch_inference.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*This* notebook provides recipes for batch inference on your videos with a trained model and the datasets used for training the model.
You need to upload your the following files from your local drive or mount the Google Drive that contains them. 
1. Custom COCO Format Dataset (e.g. my_coco_dataset.zip)
2. A trained model saved as .pth format (e.g. model_final.pth)
3. A folder contains all the videos (e.g. my_videos folder has video files like 1.mp4, 2.avi ...)

# Install Detectron2

In [ ]:
# install dependencies: 
!pip install pyyaml==5.3
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html

# Download and install Annolid

In [ ]:
!git clone --recurse-submodules https://github.com/healthonrails/annolid.git

In [ ]:
%cd annolid

In [ ]:
!pip install -e .

In [ ]:
import numpy as np
import cv2
import glob
import requests
import pandas as pd
from pathlib import Path
from annolid.annotation.keypoints import save_labels
from annolid.postprocessing.quality_control import pred_dict_to_labelme
from annolid.data.videos import frame_from_video
from annolid.inference.predict import Segmentor

# Local file system (Please skip this section if you want to use the files in your Google Drive)

## Uploading files from your local file system

`files.upload` returns a dictionary of the files which were uploaded.
The dictionary is keyed by the file name and values are the data which were uploaded.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Mount your Google Drive on this runtime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Example for unzip your COCO format dataset zip file

In [ ]:
!unzip /content/my_coco_dataset.zip -d /content

In [ ]:
DATASET_DIR = "/content/my_coco_dataset"
MODEL_PATH =  "/content/model_final.pth"
VIDEOS_FOLDER = '/content/my_videos'

#Load the predictor

In [ ]:
predictor = Segmentor(DATASET_DIR,MODEL_PATH)

# Find all the video files in the given `VIDEOS_FOLDER`

In [ ]:
video_files = glob.glob(VIDEOS_FOLDER + '/*.*')

# Batch inference for the videos

In [ ]:
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if cap.isOpened():
    cap.release()
    print("Working on video, ", video_file)
    predictor.on_video(video_file)
  else:
    print("Cannot open this file", video_file)
    cap.release()

## Downloading files to your local file system

`files.download` will invoke a browser download of the file to your local computer.


In [ ]:
from google.colab import files
tracking_csv_files = glob.glob(str(Path(DATASET_DIR).parent) + '/*mask*tracking*.csv')
for tcf in tracking_csv_files:
    files.download(tcf)

## Or saving the tracking results csv files to your Google Drive



In [ ]:
import shutil

In [ ]:
for tcf in tracking_csv_files:
    shutil.copy(tcf, '/content/drive/MyDrive/')

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')